In [1]:
# 0. Import modules
from src.instruments.vanilla_option import VanillaOption
from src.instruments.barrier_option import BarrierOption
from src.instruments.variance_swap_swaption import Variance_Swap_Swaption
from src.market_data.market import Market

from src.models.black_scholes.black_scholes_pricing import BlackScholesEngine
from src.models.pde.pde_pricing import PDEPricingEngine
from src.models.monte_carlo.monte_carlo_pricing import MonteCarloEngine

from src.valuation.valuation_request import ValuationRequest

In [2]:
# 1. Create a Market object
market = Market(
    yield_curve=None,  # a yield curve object or mock
    vol_surface_dict={},
    spot_prices={"Variance_Swap_Spot": 0.2},
)

In [3]:
# 2. Construct vanilla option Instruments
variance_swap_swaption = Variance_Swap_Swaption(
    K=0.2,
    T1=0.5,        
    T2=1.0,
)

instruments = [variance_swap_swaption]

In [6]:
# 3. Create Pricing Engines

mc_engine = MonteCarloEngine(
    interest_rate=0.01,
    volatility=0.20,
    spot_price=market.get_spot_price("Variance_Swap_Spot"),
    dividend_yield=0.0,
    n_paths=100000,
    n_steps=252,
    params={'nu':1.50, 'theta':0.312, 'k1':2.63, 'k2':0.42, 'rho':-0.7}
)

pricing_engines = [mc_engine]

In [7]:
# Print table header
print(f"{'Option Price':<15}{'Pricing Engine':<20}{'S0':<10}{'Sigma':<10}{'r':<10}{'Strike (K)':<15}{'Maturity':<10}")
print("=" * 90)

# 4. Create a Valuation Request and run valuations
for instrument in instruments:
    # Get parameters
    strike = instrument.K
    maturity = instrument.T1
    
    for pricing_engine in pricing_engines:
        # Get parameters
        pricing_engine_name = pricing_engine.__class__.__name__
        r = pricing_engine.r
        sigma = pricing_engine.sigma
        S0 = pricing_engine.S0
        
        valuation_req = ValuationRequest(instrument=instrument, pricer=pricing_engine, market=market)
        fair_value = valuation_req.run_valuation()
        
        # Print data in tabular format
        print(f"{round(fair_value, 4):<15}{pricing_engine_name:<20}{S0:<10}{sigma:<10}{r:<10}{strike:<15}{maturity:<10}")


Option Price   Pricing Engine      S0        Sigma     r         Strike (K)     Maturity  


100%|██████████| 100000/100000 [00:00<00:00, 116393.44it/s]

0.062          MonteCarloEngine    0.2       0.2       0.01      0.2            0.5       
